# On charge les données du dataset EDC


In [2]:
import duckdb
import pandas as pd
import os
import numpy as np
from pipelines.tasks.config.common import DUCKDB_FILE

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

query = """
SELECT edc_prelevements.dateprel, edc_prelevements.heureprel,edc_prelevements.inseecommuneprinc, edc_prelevements.nomcommuneprinc, edc_prelevements.cdreseau, edc_prelevements.cdreseauamont, edc_resultats.referenceprel, edc_resultats.valtraduite
FROM edc_prelevements
JOIN edc_resultats
ON edc_prelevements.referenceprel = edc_resultats.referenceprel
WHERE
(dateprel >= '2024-01-01')
"""

edc = con.sql(query)
edc = edc.df()

# On charge les données du dataset EDR


In [3]:
edr_udi_plv = pd.read_csv(
    r"C:\Users\Admin\Downloads\edr\UDI_PLV_202412.txt", delimiter=","
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_7288\612874238.py:1: DtypeWarning: Columns (0,3,4,6,11,13,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  edr_udi_plv = pd.read_csv(


In [ ]:
edr_udi_plv.sort_values(by=["dateprel"])[
    ["referenceprel", "inseecommune", "nomcommune", "dateprel"]
]

,referenceprel,inseecommune,nomcommune,dateprel
16442,6400141962,64130,BIRIATOU,2016-04-05
18611,7100072412,71576,VILLARS (LE),2016-05-25
16728,6400150278,64466,RIVEHAUTE,2017-07-03
16412,6400150317,64114,BERNADETS,2017-07-06
16677,6400150380,64422,OLORON-SAINTE-MARIE,2017-07-10
...,...,...,...,...
20144,7800244696,78172,CONFLANS-SAINTE-HONORINE,2024-12-31
8040,03100206091,31555,TOULOUSE,2024-12-31
13500,05600173024,56243,SENE,2024-12-31
7819,03100206176,31033,AUTERIVE,2024-12-31


On voit déjà qu'il y'a un soucis: le fichier censé concerner seulement 2024 contient des dates jusqu'en 2016


In [ ]:
edr = edr_udi_plv.sort_values(by=["dateprel"])[
    ["referenceprel", "inseecommune", "nomcommune", "dateprel", "cdreseau"]
]
edr = edr[edr["dateprel"] > "2024-01-01"]

In [35]:
edr

,referenceprel,inseecommune,nomcommune,dateprel,cdreseau
10547,04100105665,41295,VINEUIL,2024-01-08,041000691
2578,01000103176,10369,SEMOINE,2024-01-08,010000164
7392,02A00115194,2A351,VILLANOVA,2024-01-08,02A000701
15182,06000149736,60640,TOURLY,2024-01-09,060001217
10351,04100105668,41007,AUTHON,2024-01-10,041000582
...,...,...,...,...,...
20144,7800244696,78172,CONFLANS-SAINTE-HONORINE,2024-12-31,78000572
8040,03100206091,31555,TOULOUSE,2024-12-31,031000006
13500,05600173024,56243,SENE,2024-12-31,056000036
7819,03100206176,31033,AUTERIVE,2024-12-31,031000042


On a donc retiré 361 prélèvements qui dataient d'avant 2024 du dataset EDR


# Analyse des différences


In [36]:
query_prel = """
SELECT DISTINCT COUNT(referenceprel) AS Nb_prel
FROM edc_prelevements
"""

edc_prel = con.sql(query_prel)
edc_prel = edc_prel.df()

In [37]:
edc_prel

,Nb_prel
0,2083345


In [38]:
query_preledr = """
SELECT DISTINCT COUNT(referenceprel) AS Nb_prel
FROM edr
"""

edr_prel = con.sql(query_preledr)
edr_prel = edr_prel.df()

In [39]:
edr_prel

,Nb_prel
0,24171


On regarde combien de referenceprel de EDC sont dans EDR


La condition edc.cdreseauamont IS NULL permet de retirer les prélèvements hors UDI de la table EDC


In [40]:
query_comp1 = """
SELECT DISTINCT edc.referenceprel
FROM   edc
WHERE 
    (edc.referenceprel NOT IN (SELECT DISTINCT edr.referenceprel FROM edr))
    AND
    (edc.cdreseauamont IS NULL)
"""

comp1 = con.sql(query_comp1)
comp1 = comp1.df()

In [41]:
comp1

,referenceprel
0,08400134126
1,08400134142
2,08400134178
3,08400134247
4,08400134251
...,...
201139,02100169187
201140,01100167358
201141,09200144585
201142,07000119434


# Dans l'autre sens, on regarde les referenceprel présents dans EDR mais pas dans EDC


In [42]:
edr.columns

Index(['referenceprel', 'inseecommune', 'nomcommune', 'dateprel', 'cdreseau'], dtype='object')

In [79]:
query_comp3 = """
SELECT edr.*
FROM edr
WHERE edr.referenceprel NOT IN (SELECT DISTINCT edc.referenceprel FROM edc)
"""

comp3 = con.sql(query_comp3)
comp3 = comp3.df()

In [80]:
comp3

,referenceprel,inseecommune,nomcommune,dateprel,cdreseau
0,6500152474,65075,BAZUS-AURE,2024-02-13,65000524
1,6500152479,65440,TARBES,2024-02-15,65000709
2,8700119928,87202,VEYRAC,2024-02-21,87003376
3,8700119923,87159,SAINT-LEGER-LA-MONTAGNE,2024-02-26,87003674
4,8700119919,87091,MAISONNAIS-SUR-TARDOIRE,2024-03-04,87001545
...,...,...,...,...,...
7759,7800244697,78361,MANTES-LA-JOLIE,2024-12-31,78001988
7760,9500182794,95680,VILLIERS-LE-BEL,2024-12-31,95004073
7761,7800244700,78217,EPONE,2024-12-31,78000569
7762,7800244696,78172,CONFLANS-SAINTE-HONORINE,2024-12-31,78000572


7764 prélevements sont dans EDR et pas dans EDC, on va chercher à comprendre d'où viennent ces prélèvements


# Exemples de prélèvements manquants


In [ ]:
edc[edc["referenceprel"] == "6400215849"]

,dateprel,heureprel,inseecommuneprinc,nomcommuneprinc,cdreseau,cdreseauamont,referenceprel,valtraduite


In [ ]:
edc[edc["referenceprel"] == "06400215849"]

,dateprel,heureprel,inseecommuneprinc,nomcommuneprinc,cdreseau,cdreseauamont,referenceprel,valtraduite
16634571,2024-12-17,11h30,64339,LESTELLE-BETHARRAM,064000405,None,06400215849,0.00
16634572,2024-12-17,11h30,64339,LESTELLE-BETHARRAM,064000405,None,06400215849,320.00
16634573,2024-12-17,11h30,64339,LESTELLE-BETHARRAM,064000405,None,06400215849,0.30
16634574,2024-12-17,11h30,64339,LESTELLE-BETHARRAM,064000405,None,06400215849,0.33
16634575,2024-12-17,11h30,64339,LESTELLE-BETHARRAM,064000405,None,06400215849,0.00
16634576,2024-12-17,11h30,64339,LESTELLE-BETHARRAM,064000405,None,06400215849,0.00
16634577,2024-12-17,11h30,64339,LESTELLE-BETHARRAM,064000405,None,06400215849,0.00
16634578,2024-12-17,11h30,64339,LESTELLE-BETHARRAM,064000405,None,06400215849,0.00
16634579,2024-12-17,11h30,64339,LESTELLE-BETHARRAM,064000405,None,06400215849,0.00
16634580,2024-12-17,11h30,64339,LESTELLE-BETHARRAM,064000405,None,06400215849,0.00


Ici le prélèvement n'est pas vraiment manquant, c'est seulement un problème de formatage. On regarde si c'est aussi le cas pour d'autres prélèvements et si oui, combien:


In [ ]:
liste_prelevedr_0 = "0" + comp3["referenceprel"].to_numpy()

In [ ]:
liste_prelevedr_0

array(['06500152474', '06500152479', '08700119928', ..., '07800244700',
       '07800244696', '06500154889'], shape=(7764,), dtype=object)

In [ ]:
edr_avec0 = comp3.assign(referenceprel0=liste_prelevedr_0)

In [ ]:
query_comp5 = """
SELECT edr_avec0.*
FROM edr_avec0
WHERE 
    (edr_avec0.referenceprel NOT IN (SELECT DISTINCT edc.referenceprel FROM edc))
    AND
    (edr_avec0.referenceprel0 NOT IN (SELECT DISTINCT edc.referenceprel FROM edc))
"""

comp5 = con.sql(query_comp5)
comp5 = comp5.df()

In [96]:
comp5

,referenceprel,inseecommune,nomcommune,dateprel,cdreseau,referenceprel0
0,6500152474,65075,BAZUS-AURE,2024-02-13,65000524,06500152474
1,6500152479,65440,TARBES,2024-02-15,65000709,06500152479
2,8700119928,87202,VEYRAC,2024-02-21,87003376,08700119928
3,8700119923,87159,SAINT-LEGER-LA-MONTAGNE,2024-02-26,87003674,08700119923
4,8700119919,87091,MAISONNAIS-SUR-TARDOIRE,2024-03-04,87001545,08700119919
...,...,...,...,...,...,...
609,04000146628,40304,SOORTS-HOSSEGOR,2024-12-30,040000419,004000146628
610,8700123147,87085,LIMOGES,2024-12-30,87000601,08700123147
611,02100170632,21063,BENEUVRE,2024-12-30,021000709,002100170632
612,6600212757,66133,PALAU-DEL-VIDRE,2024-12-30,66002539,06600212757


Quand on retire les erreurs de formatage où un 0 manquait au début de referenceprel, il ne reste déjà plus que 614 prélèvements présents dans EDR mais absents de EDC, on avance


On étudie le nombre de réseaux concernés


In [ ]:
query_reseaux = """
SELECT DISTINCT GROUP_CONCAT(edr_avec0.referenceprel) AS Liste_referenceprel,GROUP_CONCAT(edr_avec0.referenceprel0) AS Liste_referenceprel0, GROUP_CONCAT(edr_avec0.inseecommune) AS Liste_inseecommune, GROUP_CONCAT(edr_avec0.nomcommune) AS nomcommune, edr_avec0.cdreseau, GROUP_CONCAT(edr_avec0.dateprel) AS Liste_dateprel
FROM edr_avec0
WHERE
    (edr_avec0.referenceprel NOT IN (SELECT DISTINCT edc.referenceprel FROM edc))
    AND
    (edr_avec0.referenceprel0 NOT IN (SELECT DISTINCT edc.referenceprel FROM edc))
GROUP BY edr_avec0.cdreseau
"""

reseaux = con.sql(query_reseaux)
reseaux = reseaux.df()

In [100]:
reseaux

,Liste_referenceprel,Liste_referenceprel0,Liste_inseecommune,nomcommune,cdreseau,Liste_dateprel
0,7800243331,07800243331,78644,VERRIERE (LA),78000543,2024-10-04
1,7300250611,07300250611,73092,CORBEL,73000583,2024-11-21
2,04100109032,004100109032,41096,GAULT-DU-PERCHE (LE),041000755,2024-12-04
3,6900167629,06900167629,69135,DEUX-GROSNES,69000362,2024-11-27
4,03500172086,003500172086,35290,SAINT-MALON-SUR-MEL,035000262,2024-12-02
...,...,...,...,...,...,...
525,03100206278,003100206278,31579,VILLARIES,031000105,2024-12-19
526,02300079589,002300079589,23057,CHATELUS-MALVALEIX,023000028,2024-11-14
527,00700204764,000700204764,07244,SAINT-JEAN-CHAMBRE,007000495,2024-12-16
528,02200236873,002200236873,22060,GAUSSON,022001253,2024-11-28


530 réseaux sont concernés par les prélèvements manquants, c'est donc assez uniformément réparti


In [102]:
comp5

,referenceprel,inseecommune,nomcommune,dateprel,cdreseau,referenceprel0
0,6500152474,65075,BAZUS-AURE,2024-02-13,65000524,06500152474
1,6500152479,65440,TARBES,2024-02-15,65000709,06500152479
2,8700119928,87202,VEYRAC,2024-02-21,87003376,08700119928
3,8700119923,87159,SAINT-LEGER-LA-MONTAGNE,2024-02-26,87003674,08700119923
4,8700119919,87091,MAISONNAIS-SUR-TARDOIRE,2024-03-04,87001545,08700119919
...,...,...,...,...,...,...
609,04000146628,40304,SOORTS-HOSSEGOR,2024-12-30,040000419,004000146628
610,8700123147,87085,LIMOGES,2024-12-30,87000601,08700123147
611,02100170632,21063,BENEUVRE,2024-12-30,021000709,002100170632
612,6600212757,66133,PALAU-DEL-VIDRE,2024-12-30,66002539,06600212757


In [ ]:
comp5[comp5["dateprel"] > "2024-10-01"]

,referenceprel,inseecommune,nomcommune,dateprel,cdreseau,referenceprel0
73,97100135781,97107,CAPESTERRE-BELLE-EAU,2024-10-02,971000140,097100135781
74,7800243227,78124,CARRIERES-SUR-SEINE,2024-10-03,78000399,07800243227
75,97100135423,97116,MORNE-A-L'EAU,2024-10-03,971001139,097100135423
76,97100135732,97113,GOSIER (LE),2024-10-04,971000027,097100135732
77,7800243331,78644,VERRIERE (LA),2024-10-04,78000543,07800243331
...,...,...,...,...,...,...
609,04000146628,40304,SOORTS-HOSSEGOR,2024-12-30,040000419,004000146628
610,8700123147,87085,LIMOGES,2024-12-30,87000601,08700123147
611,02100170632,21063,BENEUVRE,2024-12-30,021000709,002100170632
612,6600212757,66133,PALAU-DEL-VIDRE,2024-12-30,66002539,06600212757


Sur les 614 prélèvements manquants, 541 d'entre eux ont été faits après le 1er octobre 2024
